# Predict Loan Defaults

In this project I will build a machine learning model that can predict whether or not a borrower is able to pay off his or her loans. The data in this project comes from Lending Club which is a marketplace for personal loans that matches borrowers with investors. I will only analyze loans that were approved with a date range of 2007 - 2011 since most of the loans in this date range have already finished.

## Data Cleaning

In [118]:
import pandas as pd
loan_data = pd.read_csv('loans_2007.csv')
#Total number of columns
loan_data.shape[1]

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


52

In order to make this dataset more manageable, I will remove columns which:
    1. Leak information from after the loan has already been funded
    2. Don't affect the borrower's ability to pay back
    3. Require additional data in order to become helpful
    4. Redundant information
    5. Poorly formatted and need tons of additional work to become useful
    
Information about each column can be found in a data dictionary [here](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097).

In [119]:
cols_to_drop = ['id', 'member_id', 'funded_amnt', 'funded_amnt_inv',
    'grade', 'sub_grade', 'emp_title', 'issue_d', 'zip_code', 'out_prncp',
    'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
    'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
    'last_pymnt_d', 'last_pymnt_amnt']
loan_data = loan_data.drop(cols_to_drop, axis=1)
loan_data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In the above code snippit I removed 20 columns which were either contained redundant information or would not be useful to the analysis. For example columns like 'grade' and 'sub_grade' were removed because they are essentially a proxy for the interest rate, which is much easier to analyze due to its continuous nature. Colums like 'total_payment' and 'out_prncp' were removed because they referenced information from after the loan had already started to be paid off. Columns like 'id' and 'member_id' were removed since they are irrelevant to a lender's ability to pay back a loan.

### Picking a target column to predict

I will use loan_status as our target column as it tells us whether or not a loan was paid off.

In [120]:
loan_data['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

The text values in this column need to be converted to numerical values. Also, all loans which were not "Fully Paid" or "Charged Off" need to be removed as these two columns are the only ones which describe final outcomes. "Fully Paid" will then be replaced with 1 and "Charged Off" (meaning the loan was not paid) will be replaced with 0.

In [121]:
loan_data = loan_data[loan_data['loan_status'].isin(['Fully Paid', 'Charged Off'])]

replace_status = {
    "loan_status": {
        "Fully Paid": 1,
        "Charged Off": 0
    }
}

loan_data = loan_data.replace(replace_status)
loan_data['loan_status'].value_counts()

1    33136
0     5634
Name: loan_status, dtype: int64

As a final step in the feature selection process, I will remove any column that contains only 1 value as it provides no additional insight into whether a loan was paid off.

In [122]:
#Look for columns that contain only one unique value and remove
loan_columns = loan_data.columns
drop_columns = []
for column in loan_columns:
    non_null = loan_data[column].dropna().unique()
    num_true_unique = len(non_null)
    if num_true_unique == 1:
        drop_columns.append(column)
print('Dropped Columns: ')
print(drop_columns)

loan_data = loan_data.drop(drop_columns, axis=1)
print('\n')
print('Remaining Columns: ')
loan_data.columns

Dropped Columns: 
['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


Remaining Columns: 


Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'last_credit_pull_d',
       'pub_rec_bankruptcies'],
      dtype='object')

Now that I've narrowed down the number of columns, I will inspect the columns in order to determine how to handle null values

In [123]:
null_counts = loan_data.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


I will further inspect the emp_length and pub_rec_bankruptcies columns since they both have a relatively high number of null values.

In [124]:
print('emp_length value breakdown: ')
print(loan_data.emp_length.value_counts(normalize=True, dropna=False))
print('\npub_rec_bankruptcies breakdown :')
print(loan_data.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

emp_length value breakdown: 
10+ years    0.220454
< 1 year     0.116766
2 years      0.111117
3 years      0.103843
4 years      0.086717
5 years      0.082770
1 year       0.082100
6 years      0.056255
7 years      0.044313
8 years      0.037245
9 years      0.031700
NaN          0.026722
Name: emp_length, dtype: float64

pub_rec_bankruptcies breakdown :
 0.0    0.939438
 1.0    0.042456
NaN     0.017978
 2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


Since emp_length (employment length) is a very important factor in assessing the riskiness of a potential borrower and the values are pretty varied, I will keep this column but remove all rows null values.  Since 94% of values for pub_rec_bankdruptcies are 0, this column will provide little predictive value so I will simply remove it.  For the remaining columns, I will remove all rows containing null values since the number of nulls is very low for each.

In [125]:
loan_data = loan_data.drop(columns=['pub_rec_bankruptcies'], axis=1)
loan_data = loan_data.dropna(axis = 0)

## Converting Text Columns to Numerical Values

In [126]:
loan_data.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

In order to make predictions with scikit-learn, object columns containing text data will need to be converted to numerical data types.

In [127]:
object_columns_df = loan_data.select_dtypes('object')
object_columns_df.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016


1. Analyze values of 'purpose' and 'title' columns do determine which one to drop
2. Drop 'purpose' or 'title' in addition to addr_state, earliest_cr_line and last_credit_pulled_d
3. Easily convert int_rate and revol_util to numeric values
4. Get value counts for categorical columns ('home_ownership', 'verification_status', 'emp_length', 'term', and one of 'title' OR 'purpose')
5. Convert emp_length to numerical value with a mapping dictionary
6. Create dummy columns for remaining categorical columns 

In [128]:
#1. Analyze values of 'purpose' and 'title' columns do determine which one to drop
for column in ['purpose', 'title']:
    print(loan_data[column].value_counts())
    print('\n')

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64


Debt Consolidation                          2068
Debt Consolidation Loan                     1599
Personal Loan                                624
Consolidation                                488
debt consolidation                           466
Credit Card Consolidation                    345
Home Improvement                             336
Debt consolidation                           314
Small Business Loan                          298
Credit Card Loan                             294
Personal                                     290
Consolidation Loan                     

We will keep 'purpose' as the data is much easier to work with. We will drop 'title' in addition to addr_state, earliest_cr_line and last_credit_pulled_d. Addr_state refers to the borrower's state of residence which is difficult to quantify as a dummy variable since there are 50 potential values. We will also drop the two date coumns since they would require a lot of additional manipulation to benefit our analysis.

In [129]:
#2. Drop 'title' in addition to addr_state, earliest_cr_line and last_credit_pulled_d
loan_data = loan_data.drop(columns=['title', 'addr_state', 'earliest_cr_line', 'last_credit_pull_d'])

#3. Easily convert int_rate and revol_util to numeric values
loan_data["int_rate"] = loan_data["int_rate"].str.rstrip("%").astype("float")
loan_data["revol_util"] = loan_data["revol_util"].str.rstrip("%").astype("float")

#4. Get value counts for categorical columns. Will leave out 'purpose' because we already have these values
categorical_cols = ['home_ownership', 'verification_status', 'emp_length', 'term']
for column in categorical_cols:
    print(loan_data[column].value_counts())
    print('\n')

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64


Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64


10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64


 36 months    28234
 60 months     9441
Name: term, dtype: int64




In [130]:
#5. Convert emp_length to numerical values with a mapping dictionary
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loan_data = loan_data.replace(mapping_dict)

#6. Create dummy columns for remaining categorical columns
dummy_df = pd.get_dummies(loan_data[['home_ownership', 'verification_status',
    'purpose', 'term']])
#Merge dummy_df with original df and drop the original categorical columns 
loan_data = pd.concat([loan_data, dummy_df], axis=1).drop(
    columns=['home_ownership', 'verification_status', 'purpose', 'term'])
print(loan_data.columns)

Index(['loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc',
       'loan_status', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
       'purpose_educational', 'purpose_home_improvement', 'purpose_house',
       'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
       'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'term_ 36 months',
       'term_ 60 months'],
      dtype='object')


In [136]:
loan_data.head()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,...,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,...,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,...,0,0,0,0,0,0,0,1,1,0


# Making predictions

Before I start predicting whether or not a borrower can pay off their loan on time, I will select an error metric in order to determine how well the model is performing. Since the overall goal of a lender is to make money, I want my model to limit false positives (losing money by funding a loan that isn't paid) and to limit false negatives (losing money by not funding a loan that would have been paid).  Most lenders would put more of a negative weight on false positives since its a lower risk to fund fewer loans and miss out on some potential opportunities versus funding a high number of loans and actually lose money on ones that won't get paid back.

In [131]:
def calculate_error_metrics(predictions, target):
    true_pos = len(predictions[(predictions == 1) & (target == 1)])
    false_pos = len(predictions[(predictions == 1) & (target == 0)])
    true_neg = len(predictions[(predictions == 0) & (target == 0)])
    false_neg = len(predictions[(predictions == 0) & (target == 1)])
    
    #Want to minimize these
    false_pos_rate = (false_pos / (false_pos + true_neg))
    false_neg_rate = (false_neg / (false_neg + true_pos))
    
    #Want to maximize these
    true_pos_rate = (true_pos / (true_pos + false_neg))
    true_neg_rate = (true_neg / (true_neg + false_pos))
    
    print('False Positive Rate: ' + str(false_pos_rate) + '\n')
    print('False Negative Rate: ' + str(false_neg_rate) + '\n')
    print('True Positive Rate: ' + str(true_pos_rate) + '\n')
    print('True Negative Rate: ' + str(true_neg_rate) + '\n')

In [133]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression()
feature_columns = loan_data.columns.drop('loan_status')
features = loan_data[feature_columns]
target = loan_data['loan_status']

#Make predictions using 3-fold cross-validation
predictions = cross_val_predict(lr, features, target, cv=3)
#print(pd.Series(predictions))
predictions = pd.Series(predictions)

#Calculate Error Metrics
calculate_error_metrics(predictions, target)

False Positive Rate: 0.9986179664363277

False Negative Rate: 0.0015731515469323545

True Positive Rate: 0.9984268484530676

True Negative Rate: 0.0013820335636722605



Although the model was very accurate in predicting loans that would be paid off, it was wildly inaccurate in predicting loans that would be result in a default. As we can see below, part of the reason for this is because there are significantly more loans that were paid off (85.7%) in comparison to loans that resulted in a default (14.3%).

In [134]:
print(loan_data['loan_status'].value_counts(normalize=True))

1    0.856961
0    0.143039
Name: loan_status, dtype: float64


One way to update this model would be to set weights that place a heavy peanalty on incorrectly predicting defaults.  Since there are about 6 times as many 1s as 0s in the loan_status column, I will make the penalty 6 times greater for incorrectly predicting a default versus incorrectly predicting a loan that was paid off.

In [138]:
penalty = { 
    0: 10,
    1: 1 
}
lr = LogisticRegression(class_weight=penalty)
#Make predictions using 3-fold cross-validation
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

#Calculate Error Metrics
calculate_error_metrics(predictions, target)

False Positive Rate: 0.2252714708785785

False Negative Rate: 0.7721224436287363

True Positive Rate: 0.22787755637126378

True Negative Rate: 0.7747285291214215



## Conclusion

We can see that placing penalties on incorrectly predicting defaults can have a beneficial impact on the accuracy of our model. There is, however, a lot more we can do to further improve the accuracy of our predictions. One option would be to try other models like a Random Forest which are better suited for data that doesn't necessarily follow a linear relationship. Another way in which we can improve this model is to run a more in-depth analysis of the feature columns in order to figure out which factors are strongly correlated with predicting a default.

## To Do...Fine tune feature selection 